### ref: https://rateslib.readthedocs.io/en/stable/z_swpm.html

### Replicating Default SOFR

![Default SOFR](sofr_n_swap/default_sofr.png)

In [1]:
import pandas as pd
from rateslib import *

In [2]:
data = pd.DataFrame({
    "Term": ["1W", "2W", "3W", "1M", "2M", "3M", "4M", "5M", "6M", "7M", "8M", "9M", "10M", "11M", "12M", "18M", "2Y", "3Y", "4Y"],
    "Rate": [5.30111, 5.30424, 5.30657, 5.31100, 5.34800, 5.38025, 5.40915, 5.43078, 5.44235, 5.44950, 5.44878, 5.44100, 5.42730, 5.40747, 5.3839, 5.09195, 4.85785, 4.51845, 4.31705],
})

data["Termination"] = [add_tenor(dt(2023, 8, 21), _, "F", "nyc") for _ in data["Term"]]

with pd.option_context("display.float_format", lambda x: '%.6f' % x):
    print(data)

   Term     Rate Termination
0    1W 5.301110  2023-08-28
1    2W 5.304240  2023-09-05
2    3W 5.306570  2023-09-11
3    1M 5.311000  2023-09-21
4    2M 5.348000  2023-10-23
5    3M 5.380250  2023-11-21
6    4M 5.409150  2023-12-21
7    5M 5.430780  2024-01-22
8    6M 5.442350  2024-02-21
9    7M 5.449500  2024-03-21
10   8M 5.448780  2024-04-22
11   9M 5.441000  2024-05-21
12  10M 5.427300  2024-06-21
13  11M 5.407470  2024-07-22
14  12M 5.383900  2024-08-21
15  18M 5.091950  2025-02-21
16   2Y 4.857850  2025-08-21
17   3Y 4.518450  2026-08-21
18   4Y 4.317050  2027-08-23


In [3]:
sofr = Curve(
    id="sofr",
    convention="Act360",
    calendar="nyc",
    modifier="MF",
    interpolation="log_linear",
    nodes={
        **{dt(2023, 8, 17): 1.0},  # <- this is today's DF,
        **{_: 1.0 for _ in data["Termination"]},
    }
)

In [4]:
sofr_args = dict(effective=dt(2023, 8, 21), spec="usd_irs", curves="sofr")

solver = Solver(
    curves=[sofr],
    instruments=[IRS(termination=_, **sofr_args) for _ in data["Termination"]],
    s=data["Rate"],
    instrument_labels=data["Term"],
    id="us_rates",
)

data["DF"] = [float(sofr[_]) for _ in data["Termination"]]

with pd.option_context("display.float_format", lambda x: '%.6f' % x):
    print(data)

SUCCESS: `func_tol` reached after 5 iterations (levenberg_marquardt) , `f_val`: 3.116743265440467e-17, `time`: 0.0731s
   Term     Rate Termination       DF
0    1W 5.301110  2023-08-28 0.998382
1    2W 5.304240  2023-09-05 0.997208
2    3W 5.306570  2023-09-11 0.996327
3    1M 5.311000  2023-09-21 0.994862
4    2M 5.348000  2023-10-23 0.990145
5    3M 5.380250  2023-11-21 0.985856
6    4M 5.409150  2023-12-21 0.981421
7    5M 5.430780  2024-01-22 0.976721
8    6M 5.442350  2024-02-21 0.972364
9    7M 5.449500  2024-03-21 0.968194
10   8M 5.448780  2024-04-22 0.963676
11   9M 5.441000  2024-05-21 0.959670
12  10M 5.427300  2024-06-21 0.955477
13  11M 5.407470  2024-07-22 0.951395
14  12M 5.383900  2024-08-21 0.947546
15  18M 5.091950  2025-02-21 0.926160
16   2Y 4.857850  2025-08-21 0.907898
17   3Y 4.518450  2026-08-21 0.874241
18   4Y 4.317050  2027-08-23 0.842731


### Create a swap in SWPM and replicate

![Swap](sofr_n_swap/swap.png)

In [5]:
irs = IRS(
    effective=dt(2023, 11, 21),
    termination=dt(2025, 2, 21),
    notional=-100e6,
    fixed_rate=5.40,
    curves="sofr",
    spec="usd_irs",
)

### PV

In [6]:
npv = irs.npv(solver=solver)
npv

<Dual: 456,622.098604, ('sofr10', 'sofr15', 'sofr16', 'sofr17', 'sofr5', 'sofr6', 'sofr8', 'sofr9'), [-5.27190877e+02  0.00000000e+00  1.05432511e+08  1.12949455e+04
  0.00000000e+00 -9.99703673e+07  0.00000000e+00  1.39448454e+06]>

### DV01

In [7]:
dv01 = irs.delta(solver=solver).sum()
dv01

local_ccy  display_ccy
usd        usd           -11879.936805
dtype: float64

### Gamma

In [8]:
gamma = irs.gamma(solver=solver).sum().sum()

### PV01

In [9]:
pv01 = irs.analytic_delta(curve=sofr)
pv01

<Dual: -11,896.008577, ('sofr10', 'sofr16', 'sofr17', 'sofr9'), [ -176.95178153 -9937.61435225  -229.09616173 -2378.60529106]>

### Check cashflows

![Cashflows](sofr_n_swap/cashflows.png)

In [10]:
cashflows_table = irs.cashflows_table(solver=solver)
cashflows_table

local_ccy,USD
collateral_ccy,NaN
payment,
2024-02-23,-7613.744059
2025-02-25,501239.067003


In [11]:
cashflows = irs.cashflows(solver=solver)
cashflows

Type   Period  Ccy  Acc Start    Acc End    Payment Convention  \
leg1 0  FixedPeriod     Stub  USD 2023-11-21 2024-02-21 2024-02-23     act360   
     1  FixedPeriod  Regular  USD 2024-02-21 2025-02-21 2025-02-25     act360   
leg2 0  FloatPeriod     Stub  USD 2023-11-21 2024-02-21 2024-02-23     act360   
     1  FloatPeriod  Regular  USD 2024-02-21 2025-02-21 2025-02-25     act360   

             DCF     Notional        DF Collateral      Rate  Spread  \
leg1 0  0.255556 -100000000.0  0.972076       None  5.400000     NaN   
     1  1.016667 -100000000.0  0.925752       None  5.400000     NaN   
leg2 0  0.255556  100000000.0  0.972076       None  5.429793     0.0   
     1  1.016667  100000000.0  0.925752       None  4.906978     0.0   

            Cashflow           NPV  FX Rate       NPV Ccy  
leg1 0  1.380000e+06  1.341464e+06      1.0  1.341464e+06  
     1  5.490000e+06  5.082380e+06      1.0  5.082380e+06  
leg2 0 -1.387614e+06 -1.348865e+06      1.0 -1.348865e+06  
     1 -4.988761e+06 -4.618357e+06      1.0 -4.618357e+06